In [ ]:
import pandas as pd

merged = pd.read_parquet('../cache/merged.parquet')

In [ ]:
merged.dtypes

In [ ]:
merged.columns

In [ ]:
numerical_columns = merged.select_dtypes(include=['int64', 'float64']).columns.tolist()
for col in numerical_columns:
    print(col, merged[col].nunique())

In [ ]:
categorical_columns = merged.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_columns:
    print(col, merged[col].nunique())

In [ ]:
# one hot encode the categorical columns
merged_encoded = pd.get_dummies(merged, columns=['QuadClass'])

In [ ]:
# label encode the categorical columns
from sklearn.preprocessing import LabelEncoder

columns_to_encode =['Actor1Country', 'Actor1GeoCountry', 'Actor1Type', 'Actor1TypeGeneral', 'Actor2Country', 'Actor2GeoCountry', 'Actor2Type','Actor2TypeGeneral', 'ActionCountry', 'EventType']

le = LabelEncoder()
for col in columns_to_encode:
    merged_encoded[col] = le.fit_transform(merged_encoded[col])

In [ ]:
merged_encoded.head()

In [ ]:
merged_encoded.dtypes

In [ ]:
merged_encoded.to_parquet("../cache/encoded.parquet")